# Parte 02

In [ ]:
import os
import sys

# Necessário mudar o diretório de trabalho para o nível mais acima
current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)

# Inserir aqui o caminho da biblioteca do savime
py_savime_path =  '/home/daniel/PycharmProjects/intelipetro/savime'
sys.path.insert(0, py_savime_path)

# Inserir aqui o caminho do arquivo de dados: um json contendo informações a respeito 
# da partição de x e y utilizada na parte 01.
data_fp = 'saved_models/data.json'

# Configuração do host e porta em que o SAVIME está escutando
host = '127.0.0.1'
port = 65000

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from sklearn.metrics import mean_squared_error
import re

from schema.tar import Tar, ImplicitTarDimensionSpecification, TarAttributeSpecification
from savime.datatype import SavimeSupportedTypes
from schema.schema import IntervalRange, IndexRange
from schema.dataset import FileDataset
from schema.subtar import OrderedSubTarDimensionSpecification, SubTarAttributeSpecification, SubTar
from savime.client import Client
from misc.command_runner import CommandRunner
from util.converter import DataVariableBlockConverter
from util.data_variable import DataVariableBlockOps

from src.predictor_consumer import PredictionConsumer

In [ ]:
with open(data_fp, 'r') as _in:
    data = json.load(_in)

output_dir = data['output_dir']
x_fp = os.path.join(output_dir, data['x_file_name'])
y_fp = os.path.join(output_dir, data['y_file_name'])

x = np.load(x_fp)
y = np.load(y_fp)
num_observations, num_features = x.shape
y_num_columns = 1 if len(y.shape) == 1 else y.shape[1]

x_c_fp = os.path.join(output_dir, 'x_data')
y_c_fp = os.path.join(output_dir, 'y_data')

# Salvar os arrays numpy em um formato legível ao SAVIME, isto é, como arrays (row-wise) e 
# sem metadados do numpy.
x.astype('float64').tofile(x_c_fp)
y.astype('float64').tofile(y_c_fp)

In [ ]:
# Definição do datasets a serem utilizados:
x_dataset = FileDataset(name='x', file_path=x_c_fp, data_type=SavimeSupportedTypes.DOUBLE,
                        is_in_savime_storage=False, num_columns=num_features)
y_dataset = FileDataset(name='y', file_path=y_c_fp, data_type=SavimeSupportedTypes.DOUBLE,
                        is_in_savime_storage=False)

# O comando gerado será:
print(x_dataset.create_query_str(), y_dataset.create_query_str(), sep='\n')    

In [ ]:
# Definição do Tar a ser empregado:
index = ImplicitTarDimensionSpecification(name='index',
                                          data_type=SavimeSupportedTypes.INT32,
                                          interval=IntervalRange(1, num_observations, 1))

x_attr = TarAttributeSpecification(name='x', data_type=SavimeSupportedTypes.DOUBLE,
                                   num_columns=num_features)

y_attr = TarAttributeSpecification(name='y', data_type=SavimeSupportedTypes.DOUBLE,
                                   num_columns=y_num_columns)

tar = Tar(name='example', dimension_specification=[index], attribute_specification=[x_attr, y_attr])

# O comando gerado será:
print(tar.create_query_str())

In [ ]:
# Carregamento do SubTar:

sub_tar_index = OrderedSubTarDimensionSpecification(dimension=index,
                                                    index_range=IndexRange(start=1, stop=num_observations,
                                                                           is_physical=False))
sub_tar_x = SubTarAttributeSpecification(attribute=x_attr, dataset=x_dataset)
sub_tar_y = SubTarAttributeSpecification(attribute=y_attr, dataset=y_dataset)

sub_tar = SubTar(tar=tar, dimension_specification=[sub_tar_index], attribute_specification=[sub_tar_x, sub_tar_y])

# O comando gerado será:
print(sub_tar.load_query_str())

**Para executar os passos a seguir é necessário que haja um servidor SAVIME escutando no host e porta definidos anteriormente.**

In [ ]:
# 1. Conexão é aberta e fechada com o SAVIME (contexto with)
# 2. Criação de um objeto de execução de comandos vinculado à conexão criada.
# 3. a) Criação dos datasets
#    b) Criação do subtar
#    c) Carregamento dos datasets por meio de um subtar

with Client(host=host, port=port) as client:
    command_runner = CommandRunner(client)
    
    command_runner.create(x_dataset)
    command_runner.create(y_dataset)
    command_runner.create(tar)
    command_runner.load(sub_tar)

In [ ]:
# Os objetos abaixo são utilizados para converter do container mais genérico DataVariableBlock
# para xarray e pandas. DataVariableBlocks contêm dois atributos: dims e attrs. Cada um desses contém 
# uma dicionário do tipo nome_array: array, onde array é um numpy array.

xarray_converter = DataVariableBlockConverter('xarray')
pandas_converter = DataVariableBlockConverter('pandas')

# Efetuar select no SAVIME a fim de verificar se o TAR for criado de forma adequada
with Client(host=host, port=port) as client:
    responses = client.execute(f'SELECT({tar.name})')
        
# Em geral, o retorno do SAVIME a uma consulta é dado por subtar. Ou seja, se o tar contem n subtars então
# a variável responses acima será uma lista com n DataVariableBlocks. Abaixo, eles são concatenados.
data_variable_block = DataVariableBlockOps.concatenate(responses)

# E abaixo convertidos
xdataset_response = xarray_converter(data_variable_block)
pandas_response = pandas_converter(data_variable_block)

In [ ]:
# Como xarray representa arrays como matrizes multidimensionais densas e SAVIME é mais genérico, 
# abarcando matrizes esparsas, é preciso manter uma máscara _mask_ identificando se determinado
# elemento na matriz está presente ou não.
print(xdataset_response)
print()
print(pandas_response)

In [ ]:
# Checando se a resposta é correta
print('pandas ok:', np.allclose(x, pandas_response['x'].values.reshape(x.shape)))

print('xarray ok:', 
np.allclose(x,
xdataset_response['x'].values[xdataset_response['_mask_']].reshape(x.shape)))

**Para executar os passos abaixo é nessário que o servidor de modelos esteja rodando.**
Execute o comando abaixo:
`tensorflow_model_server --rest_api_port=8501 --model_config_file=ARQUIVO_DE_MODELOS` 
Note que você deve trocar ARQUIVO_DE_MODELOS pelo caminho do arquivo no qual os modelos foram registrados. Esse arquivo é o `models.config` dentro da pasta `saved_models`.

In [ ]:
tfx_host = 'localhost'
tfx_port = 8501
model_names = data['splits'].keys()

# Ordena os nomes 0, 1, 2, 3,...
def get_int_part(n):
    return int(re.findall('\d+', n)[0])
model_names = sorted(model_names, key=lambda x: get_int_part(x))

mse_array = np.load(os.path.join(output_dir, 'mse_array.npy')).ravel()

for i, model_name in enumerate(model_names):
    # O objeto abaixo é vinculado ao model (dado por model_name)
    model_predictive_service = PredictionConsumer(host=tfx_host, port=tfx_port, model_name=model_name)
    # Abaixo é enviado o array x como consulta preditiva, e retornado y_hat
    y_hat = model_predictive_service.predict(x)
    
    model_mse = mean_squared_error(y, y_hat)
    print(f'O MSE de {model_name}: {model_mse:4f} (dado por tfx) e {mse_array[i]:4f} (computado anteriormente).')

# Abaixo não está funcionando

In [ ]:
model = model_names[0]

with Client(host=host, port=port) as client:
    command_runner = CommandRunner(client)
    command_runner.register_model(model_name=model, model_tar=tar.name,
                                  target_attribute=x_attr.name,
                                  dim_specification={index.name: 1000})
    command_runner.predict(tar=tar.name, model_name=model, target_attribute=x_attr.name)